In [ ]:
import os
import math
import numpy as np
import cupy as cp
import matplotlib.pyplot as plt
import cupyx.scipy.sparse as csp
from IPython.display import display
from PIL import Image
import package.myUtil as myUtil

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
n = 128
DO_THIN_OUT = False
SAVE_AS_SPARSE = True
# DATA_PATH = "../../OneDrive - m.titech.ac.jp/Lab/data"
DATA_PATH = "../data"
IMG_NAME = "hadamard"
CAP_DATE = "241114"
EXP_DATE = "241202"
DIRECTORY = f"{DATA_PATH}/{EXP_DATE}"
SETTING = f"gf"

if not os.path.exists(DIRECTORY):
    os.makedirs(DIRECTORY)
if not os.path.exists(DIRECTORY + "/systemMatrix"):
    os.makedirs(DIRECTORY + "/systemMatrix")

In [ ]:
use_list = myUtil.get_use_list(n*n, 1.0)
G = myUtil.images2matrix(f"{DATA_PATH}/{IMG_NAME}{n}_cap_{CAP_DATE}/", use_list, thin_out=DO_THIN_OUT)
F = myUtil.images2matrix(f"{DATA_PATH}/{IMG_NAME}{n}_input/", use_list).astype(cp.int8)
M, K = G.shape
N, K = F.shape
print("G shape:", G.shape, "F shape:", F.shape, "M=", M, "N=", N, "K=", K)
print("G max:", G.max(), "G min:", G.min(), "F max:", F.max(), "F min:", F.min())

black = myUtil.calculate_bias(M, DATA_PATH, CAP_DATE)
B = cp.tile(black[:, None], K)

G = G - B

white_img = Image.open(f"{DATA_PATH}/capture_{CAP_DATE}/White.png").convert("L")
white = (cp.asarray(white_img) / 255).astype(cp.float32)
if DO_THIN_OUT:
    white = white[::2, ::2].ravel() - black
else:
    white = white.ravel() - black
H1 = cp.tile(white[:, None], K)

F_hat = 2 * F - 1
G_hat = 2 * G - H1
del F, G, H1

In [ ]:
H = G_hat @ F_hat.T / N
# H[:, 0] = H[:, 1]
print("H shape:", H.shape)

In [ ]:
if SAVE_AS_SPARSE:
    # Thresholding
    threshold = 1e-4
    H = cp.where(cp.abs(H) < threshold, 0, H)
    H_csp = csp.csr_matrix(H)
    print(f"shape: {H_csp.shape}, nnz: {H_csp.nnz}({H_csp.nnz / H_csp.shape[0] / H_csp.shape[1] * 100:.2f}%)")
    H_np = {
        "data": cp.asnumpy(H_csp.data),
        "indices": cp.asnumpy(H_csp.indices),
        "indptr": cp.asnumpy(H_csp.indptr),
        "shape": H_csp.shape
    }
    np.savez(f"{DIRECTORY}/systemMatrix/H_matrix_{SETTING}.npz", **H_np)
    print(f"Saved {DIRECTORY}/systemMatrix/H_matrix_{SETTING}.npz")
    myUtil.plot_sparse_matrix_cupy(H_csp, row_range=(5500, 6000), col_range=(4500, 5000), markersize=1)
else:
    cp.save(f"{DIRECTORY}/systemMatrix/H_matrix_{SETTING}.npy", H)
    print(f"Saved {DIRECTORY}/systemMatrix/H_matrix_{SETTING}.npy")

In [ ]:
SAMPLE_NAME = "Cameraman"
sample_image = Image.open(f"{DATA_PATH}/sample_image{n}/{SAMPLE_NAME}.png").convert("L")
sample_image = cp.asarray(sample_image).flatten() / 255

m = int(math.sqrt(M))
FILENAME = f"{SAMPLE_NAME}_{SETTING}.png"
if SAVE_AS_SPARSE:
    Hf = H_csp @ sample_image + black
else:
    Hf = H @ sample_image + black
Hf = cp.asnumpy(Hf.reshape(m, m))
print("Hf shape:", Hf.shape)

Hf_pil = Image.fromarray((Hf * 255).astype(np.uint8), mode="L")
Hf_pil.save(f"{DIRECTORY}/{FILENAME}", format='PNG')
print(f"Saved {DIRECTORY}/{FILENAME}")
display(Hf_pil)

plt.imshow(Hf, cmap='gray', interpolation='nearest')
plt.colorbar()
plt.title('Grayscale Heatmap')
plt.show()